In [12]:
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta

In [13]:
df = pd.read_csv("../processed-data.csv")
df

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred,MonteCarlo_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
215,2027-12-01,NaN,NaN,NaN,NaN,NaN,NaN,431.275734
216,2028-01-01,NaN,NaN,NaN,NaN,NaN,NaN,431.451643
217,2028-02-01,NaN,NaN,NaN,NaN,NaN,NaN,431.780191
218,2028-03-01,NaN,NaN,NaN,NaN,NaN,NaN,431.664951


In [14]:
def evaluateModel(df, predictedColumnName):
    validDf = df.dropna(subset=["Value", predictedColumnName])
    
    if len(validDf) == 0:
        return {
            "mse": np.nan,
            "rmse": np.nan,
            "mae": np.nan,
            "mape": np.nan
        }
    
    actual = validDf["Value"]
    predicted = validDf[predictedColumnName]
    
    mse = np.mean((actual - predicted) ** 2)
    
    rmse = np.sqrt(mse)
    
    mae = np.mean(np.abs(actual - predicted))
    
    nonZeroActual = actual[actual != 0]
    nonZeroPredicted = predicted[actual != 0]
    
    if len(nonZeroActual) > 0:
        mape = np.mean(np.abs((nonZeroActual - nonZeroPredicted) / nonZeroActual)) * 100
    else:
        mape = np.nan
    
    return {
        "mse": mse,
        "rmse": rmse,
        "mae": mae,
        "mape": mape
    }

In [ ]:
def predictWithMonteCarlo(df, nFuture=12, nSim=1000, windowSize=12):
    dfCopy = df.copy()
    
    values = dfCopy['Value'].dropna().values
    changes = values[1:] - values[:-1]
    meanChange = changes.mean()
    stdChange = changes.std()
    
    monteCarloPreds = [np.nan] * len(dfCopy)
    
    for i in range(windowSize, len(values)):
        sims = []
        for _ in range(nSim):
            simValue = values[i-windowSize]
            for j in range(windowSize):
                simValue += np.random.normal(meanChange, stdChange)
            sims.append(simValue)
        pred = np.mean(sims)
        
        valueIndex = dfCopy['Value'].dropna().index[i]
        monteCarloPreds[valueIndex] = pred
    
    dfCopy['MonteCarlo_Pred'] = monteCarloPreds
    
    lastValue = values[-1]
    futurePreds = []
    
    for i in range(nFuture):
        sims = []
        for _ in range(nSim):
            simValue = lastValue
            # simulate forward for windowSize + i steps
            for j in range(windowSize + i):
                simValue += np.random.normal(meanChange, stdChange)
            sims.append(simValue)
        pred = np.mean(sims)
        futurePreds.append(pred)
    
    futureRows = dfCopy[dfCopy['Value'].isna() & dfCopy['MovingAvg'].notna()].index
    
    for i, idx in enumerate(futureRows[:len(futurePreds)]):
        dfCopy.loc[idx, 'MonteCarlo_Pred'] = futurePreds[i]
    
    # evaluate model performance
    print(evaluateModel(dfCopy, "MonteCarlo_Pred"))
    
    return dfCopy

In [16]:
df = predictWithMonteCarlo(df, nFuture=12, nSim=1000)
df

{'mse': np.float64(0.5204972772258352), 'rmse': np.float64(0.7214549724174303), 'mae': np.float64(0.5853729465465972), 'mape': np.float64(0.1436259803222661)}


,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred,MonteCarlo_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
215,2027-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,2028-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,2028-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,2028-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Export it
outPath = "../processed-data.csv"
df.to_csv(outPath, index=False)
df

,Date,Value,MovingAvg,PercAvg_Pred,ExpSmooth,RMA,RMA_Pred,MonteCarlo_Pred
0,2010-01-01,388.91,NaN,407.037443,388.91000,NaN,NaN,NaN
1,2010-02-01,390.41,NaN,407.902507,389.36000,NaN,NaN,NaN
2,2010-03-01,391.37,NaN,408.657562,389.96300,NaN,NaN,NaN
3,2010-04-01,392.67,NaN,410.147672,390.77510,NaN,NaN,NaN
4,2010-05-01,393.21,NaN,409.564129,391.50557,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
215,2027-12-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
216,2028-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
217,2028-02-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
218,2028-03-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
